# **Proyecto final de Ciencia de Datos II - Coder House**
# ~ Modelo de predicción de precios de vehículos ~
Estudiante: **Gonzalo Leonel Gramajo**  
Comisión: **75690**  
Documento: **40441349**  
Año: **2025**

___________________________________________________________________________________________
## 1. **INTRODUCCIÓN**

Este notebook corresponde al trabajo final del curso "CIENCIA DE DATOS II" de Coder House.  
Está basado en un dataset de ventas de autos rescatado desde Keegle. Este conjunto de datos incluye todas las publicaciones de vehículos usados ​​dentro de los Estados Unidos en Craigslist.com.
Es importante resaltar que se cuenta con 426 mil lineas y el archivo ocupa 1.45 GB de almacenamiento, por lo que un archivo de tal tamaño no se peude subir a GitHub. Dado esto, el archivo .csv se puede encontrar disponible en Google Drive y Kaggle.

### **Dataset**
Used Cars Dataset - Vehicles listings from Craigslist.org.  
Enlace web a Kaggle: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data  
Enlace web a Google Drive: https://drive.google.com/file/d/1uQ_YhqBimI46j5W-EgSwkjZvpFt87ejt/view?usp=sharing

### **Objetivo general**
*Lograr un modelo predictivo que logre inferir automáticamente el precio de los vehículos acorde a las caracteristicas más relevantes de los mismos.*

### **Objetivos específicos**
* *Encontrar las variables de correlación principales con el precio del vehículo.*
* *Lograr un primer modelo predictivo y evaluar su rendimiento.*
* *Generar un modelo predictivo optimizado con métricas maximizadas.*

### **Preguntas de interés**

1. ¿Los autos con menos kilometraje (odometer) tienden a ser más valorados?  
1.1. ¿Cuál es la correlación entre el kilometraje (odometer) y el precio del vehículo?  
1.2. ¿Existe un umbral de kilometraje a partir del cual el valor del vehículo cae significativamente?  
  
2. ¿Cómo varía el precio de los vehículos según su marca/fabricante?  
2.1. ¿Qué marcas presentan mayor dispersión de precios en sus publicaciones?  
2.2. ¿Existen marcas cuyos vehículos están consistentemente sobrevalorados o subvalorados respecto a la media general?    
  
3. ¿Los vehículos con año de fabricación menor, se venden exponencialmente mas caros?  
3.1. ¿Cómo varía el precio medio de los vehículos según el año de fabricación?  
3.2. ¿Se observa una depreciación lineal o no lineal del precio con el paso de los años?  
   
4. ¿Los Fabricantes con más publicaciones, son los que se acercan más a la media del precio?   
4.1. ¿Los fabricantes con más publicaciones son de marcas nacionales?   
4.2. ¿Los fabricantes nacionales tienen precios más bajos que otras marcas?   

___________________________________________________________________________________________
## 2. **CARGA Y VERIFICACIÓN DE DATOS**

### 2.1. **CARGA DE DATOS E IMPORTACIONES**

In [ ]:
# Herramientas principales
import pandas as pd
import numpy as np
# Módulos de visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Carga el archivo CSV en un DataFrame de pandas
file_path = './vehicles.csv' # ruta al archivo CSV
df = pd.read_csv(file_path) # leer el archivo CSV y conseguir un dataframe

Mostrar  las primeras 5 lineas del dataframe cargao para tener un pantallazo de que se trata.

In [ ]:
df.head() # mostrar los primeros 5 valores

A simple vista y rápidamente se puede observar una gran cantidad de información que no es relevante para el objetivo, como ser las URLs. También se ve muchos NaN.

### 2.2. **REVISIÓN GENERAL Y LIMPIEZA INICIAL**

In [ ]:
df.info()
print("\n Cantidad de valores nulos por columna: \n")
df.isnull().sum()

**Revisar si year tiene decimales**. En la "info" del dataframe, se puede ver que year es de tipo float. Esto es algo extraño ya que suelen ser valores enteros, por lo tanto, se debe averiguar por qué esto es así. Puede ser que existan valores decimales o no.

In [ ]:
# Función para verificar si hay decimales en una columna
def tiene_decimales(serie):
    return (serie % 1 != 0).any()

print("¿'year' tiene decimales?", tiene_decimales(df['year'].dropna()))


Se verifica que year no tiene desimales, por lo tanto se cambia su tipo de dato a entero (int):

In [ ]:
if not tiene_decimales(df['year'].dropna()):
    df['year'] = df['year'].astype('Int64')  # acepta nulos (NaN)

**Análisis de posting_date:**  
Para tener una certeza de que rango de fechas se estan por utilizar en el modelo:

In [ ]:
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype

# Asegurarse de que 'posting_date' sea datetime, con zona horaria UTC
df['posting_date'] = pd.to_datetime(df['posting_date'], errors='coerce', utc=True)

# Convertir de datetime con zona horaria a sin zona horaria
df['posting_date'] = df['posting_date'].dt.tz_convert(None)

# Verificar tipo final
print(df['posting_date'].dtype)

# Análisis
print("Fecha mínima:", df['posting_date'].min())
print("Fecha máxima:", df['posting_date'].max())
print("Cantidad de fechas únicas:", df['posting_date'].nunique())


Se comprueba que la cantidad de publicaciones abarca aproximadamente un mes. Esto es un periodo relativamente corto, más para un mercado como el estadounidense que tiene una inflación muy baja, como así también estabilidad en tasas de interés y disponibilidad de crédito; por lo que se asumirá que un vehículo publicado al inicio del rango de fechas  de "posting", se encuentra en una situación económica casi igual que al final del rango.
Asumiendo esto, se procede decide eliminar el campo "posting_date".

**Eliminación de columnas irrelevantes:**  
Se eliminan los campos:
1. los que tienen URLs, textos largos o son todas nulls. En este ultimo caso, solo se trata de **country**, pero se sabe que todos los datos son de vehículos publicados en los **Estados Unidos**.  
2. **VIN** que sería como el registro del automotor, ya que simplemente es un identificador que no es relevante para el análisis y modelado.  
3. **lat** y **long** que serían la latitud y longitud en donde se realizan los posteos, ya que esto es irrelevante para el modelado y en todo caso, tambien se cuenta con el campo *state (estado)* que permitiría hacer una división de las publicaciones más interesante y sí sería *relevante* para el modelado. 
4. **region** porque ya se tiene un dato muy parecido que es el estado (state) y de hecho, la región es una división hecha por Criglist.com para poder mostrar los anuncios correctamente.
5. **posting_date** por los resultados de su análisis.
6. **id** porque es simplemente un identificador único, sin valor predictivo. Si lo dejás, el modelo puede memorizar registros, generando overfitting sin aportar nada real.

In [ ]:
df.drop(columns=['url', 'region_url', 'image_url', 'description', 'VIN', 'county', 'lat', 'long', 'region', 'posting_date', 'id'], inplace=True)

In [ ]:
df.info()

___________________________________________________________________________________________
## 3. **ANÁLISIS DE CARACTERISTICAS**

### 3.1. **VERIFICAR VALORES NULOS Y PERDIDOS**

In [ ]:
# Análisis de valores nulos en el DataFrame
def resumen_nulos(df):
    total_nulos = df.isnull().sum()
    porcentaje_nulos = (total_nulos / len(df)) * 100
    resumen = pd.DataFrame({
        'Nulos': total_nulos,
        '% Nulos': porcentaje_nulos
    })
    resumen = resumen[resumen['Nulos'] > 0]
    resumen = resumen.sort_values(by='% Nulos', ascending=False)
    return resumen

# Mostrar el resumen
resumen_nulos(df)

### 3.2. **VALORES NUMÉRICOS**

| Columna      | % Nulos | Descripción              | Acción correctiva          |
| ------------ | ------- | ------------------------ | -------------------------- |
| **odometer** | 1.03%   | Kilometraje del vehículo | Completar con **mediana**  |
| **year**     | 0.28%   | Año de fabricación       | Completar con **mediana**  |

Justificación:
* Ambas columnas tienen nulos en un porcentaje muy bajo.
* Dado que son variables continuas, la mediana es la mejor opción para evitar sesgos por outliers.

### 3.3. **VALORES CATEGÓRICOS**

| Columna           | % Nulos | Descripción                                | Importancia para el modelo | Acción correctiva sugerida                          |
| ----------------- | ------- | ------------------------------------------ | -------------------------- | --------------------------------------------------- |
| **manufacturer**  | 4.13%   | Marca del fabricante                       | Muy alta                   | **Eliminar filas con nulos** (clave para el modelo) |
| **model**         | 1.24%   | Modelo específico                          | Alta                       | Completar con **"unknown"**                         |
| **condition**     | 40.78%  | Estado del vehículo                        | Alta                       | Completar con **"unknown"**                         |
| **cylinders**     | 41.62%  | Cantidad de cilindros                      | Media-alta                 | Completar con **"unknown"**                         |
| **drive**         | 30.59%  | Tracción (FWD, RWD, 4WD)                   | Media                      | Completar con **"unknown"**                         |
| **paint\_color**  | 30.50%  | Color de la pintura                        | Baja                       | Completar con **"unknown"**                         |
| **type**          | 21.75%  | Tipo de vehículo (SUV, sedan, truck, etc.) | Media                      | Completar con **"unknown"**                         |
| **title\_status** | 1.93%   | Estado del título (clean, salvage, etc.)   | Alta                       | Completar con **moda**                              |
| **fuel**          | 0.71%   | Tipo de combustible                        | Alta                       | Completar con **moda**                              |
| **transmission**  | 0.60%   | Tipo de transmisión                        | Alta                       | Completar con **moda**                              |

Justificación:
* Las columnas con menos del 5% de nulos (como fuel, transmission, title_status) son candidatas fuertes a completarse con la moda porque el valor más frecuente suele ser representativo.
* Las columnas con más del 20% de nulos y que son categóricas no ordinales (condition, cylinders, drive, paint_color, type, model) pueden completarse con un valor neutro como "unknown" para no perder registros.

___________________________________________________________________________________________
## 4. **MANEJO DE VALORES FALTANTES Y ANÓMALOS**

### 4.1. **MANEJO DE VALORES FALTANTES**

In [ ]:
# Eliminar la columna "size" con demasiados nulos
df.drop(columns=['size'], inplace=True)

# Eliminar filas con manufacturer nulo (muy pocas)
df = df[df['manufacturer'].notnull()]

# Completar con 'unknown' en columnas categóricas relevantes
cols_unknown = ['cylinders', 'condition', 'drive', 'paint_color', 'type', 'model']
df[cols_unknown] = df[cols_unknown].fillna('unknown')

# Completar con moda las columnas categóricas
cols_moda = ['title_status', 'fuel', 'transmission']
for col in cols_moda:
    moda = df[col].mode()[0]
    df[col] = df[col].fillna(moda)

# Completar numéricas con mediana las columnas numéricas
cols_mediana = ['odometer', 'year']
for col in cols_mediana:
    mediana = df[col].median()
    df[col] = df[col].fillna(mediana)

Una vez realizada la depuración y manejo de valores faltantes, conviene hacer un .info para verificar como quedaron las columnas y su integridad.

In [ ]:
df.info()

### 4.2. **DETECCIÓN Y MANEJO DE OUTLIERS**

Primero se debe detectar los outliers. Estos son facilmente visualizables con un boxplot.

In [ ]:
sns.boxplot(x=df['price'])
plt.show()

sns.boxplot(x=df['odometer'])
plt.show()

sns.boxplot(x=df['year'])
plt.show()

Con respecto a la columna price, es imprescindible aplicar la eliminación de outliers fijando un umbral para precios fuera del rango $500 - $2.000.000. Esto es porque:  
| Corte               | Motivo                                                                      |
| ------------------- | --------------------------------------------------------------------------- |
| `price < 1000`       | Precios demasiado bajos → autos posiblemente no reales o datos mal cargados |
| `price > 200,000` | Excesivamente altos para el contexto de Craigslist (vehículos normales)     |

In [ ]:
print("Antes del filtrado:", len(df))
df = df[(df['price'] >= 1000) & (df['price'] <= 200_000)]
print("Después del filtrado:", len(df))

Con respecto a year y odometer, se realiza el sigeuinte histograma para relacionarlos y luego truncar los datos de manera más justificada.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df['year'], df['odometer'], alpha=0.1)
plt.title("Relación entre Año de fabricación y Kilometraje")
plt.xlabel("Año (year)")
plt.ylabel("Kilometraje (odometer)")
plt.grid(True)
plt.tight_layout()
plt.show()

Como se puede ver en el histograma, existen vehículos con el odometro por encima de 4 millones de kilómetros... esto es irreal. Como así tambien se ve autos muy viejos, como por ejemplo de 1940 para atrás.  

Cortes a realizar:
- Eliminar vehículos con kilometraje mayor a 4 millones
- Eliminar vehículos anteriores a 1940 (inclusive)

In [ ]:
print("Antes del filtrado:", len(df))
df = df[(df['odometer'] <= 1_000_000) & (df['year'] >= 1940)]
print("Después del filtrado:", len(df))

___________________________________________________________________________________________
## 5. **INGENIERÍA DE CARACTERÍSTICAS**

### 5.1. **ANÁLISIS DE CARDINALIDAD DE VARIABLES CATEGÓRICAS**

Antes de hacer una codificación, es necesario saber la situación de las variables categrícas, para eso es menester usar el siguiente gráfico:

In [ ]:
# Seleccionar columnas categóricas
cat_cols = df.select_dtypes(include='object').columns
# Contar valores únicos por columna
unique_counts = df[cat_cols].nunique().sort_values(ascending=False)

# Mostrar tabla
print(unique_counts)

# Paso 4: Graficar
plt.figure(figsize=(10, 6))
sns.barplot(x=unique_counts.values, y=unique_counts.index)
plt.title('Cantidad de valores únicos por columna categórica')
plt.xlabel('Cantidad de valores únicos')
plt.ylabel('Columnas categóricas')
plt.tight_layout()
plt.show()


Como se puede observar en el diagrama de barras, model tiene una gran cardinalidad. Por lo tanto, se debe agrupar los valores por frecuencias:

In [ ]:
top_models = df['model'].value_counts().nlargest(100).index
df['model_grouped'] = df['model'].apply(lambda x: x if x in top_models else 'other')

In [ ]:
# Contar la cantidad de vehículos por modelo agrupado
model_counts = df['model_grouped'].value_counts().sort_values(ascending=False)

# Mostrar los primeros 100 modelos
print(model_counts.head(100))

# Filtrar el top 100
top_100_models = model_counts.head(100)

# Gráfico
plt.figure(figsize=(12, 18))  # Más alto para que no se amontonen las etiquetas
sns.barplot(x=top_100_models.values, y=top_100_models.index)


Una vez que se logró tratar a la característica model, se procede a analizar las demás caracteristicas que tienen mucha menos cardinalidad, por lo que visualmente se puede ver si los datos son consistentes:

In [ ]:
# Lista de columnas a analizar
cat_cols = ['manufacturer', 'type', 'paint_color', 'cylinders', 'condition', 
            'title_status', 'fuel', 'drive', 'transmission']

# Mostrar valores únicos y frecuencias por columna
for col in cat_cols:
    print(f"\n📌 Columna: {col}")
    print("-" * 40)
    print(df[col].value_counts(dropna=False))


### 5.2. **NUEVAS CARACTERÍSTICAS DERIVADAS**

Se generarán las siguientes nuevas características:

| Nueva columna       | Cómo se calcula                                                          | Justificación                                 |
| ------------------- | ------------------------------------------------------------------------ | --------------------------------------------- |
| `vehicle_age`       | `vehicle_age = 2021 - year` *(ya que posting\_date era abril/mayo 2021)* | Edad del vehículo es mejor que año bruto      |
| `cylinders_num`     | Extraer número de `cylinders` (usar regex o split)                       | Transforma texto a número utilizable          |
| `is_automatic`      | `1` si transmisión es automática, `0` si manual                          | Más directo para modelo                       |
| `is_clean_title`    | `1` si `title_status == "clean"`, `0` si no                              | Título limpio suele aumentar el precio        |


In [ ]:
# 1. VEHICLE AGE (edad del vehículo)
df['vehicle_age'] = 2021 - df['year']

# 2. CYLINDERS_NUM (extracción e imputación con mediana)
df['cylinders_num'] = df['cylinders'].str.extract(r'(\d+)').astype(float)
mediana_cyl = df['cylinders_num'].median()
df['cylinders_num'] = df['cylinders_num'].fillna(mediana_cyl)

# 3. IS_AUTOMATIC
def map_transmission(value):
    if value == 'automatic':
        return 1
    elif value == 'manual':
        return 0
    else:  # 'other' o desconocido
        return -1

df['is_automatic'] = df['transmission'].apply(map_transmission)

# 4. IS_CLEAN_TITLE
df['is_clean_title'] = df['title_status'].apply(lambda x: 1 if x == 'clean' else 0)

# VERIFICACIÓN DE LOS CAMBIOS
df.info()

### 5.3. **CODIFICACIÓN DE VARIABLES CATEGÓRICAS**

Se utilizará One-Hot Encoding para todas las variables categóricas porque son nominales, no ordinales.  
Por otro lado, se eliminarán las columnas transmission, model y cylinders porque ya se ubtuvo su verisón no categórica (is_automatic, cylinders_num). En el caso de model, se obtuvo model_group que debe ser codificada.

In [ ]:
# Eliminar columnas innecesarias que ya fueron codificadas
df.drop(columns=['transmission', 'cylinders', 'model'], inplace=True)

cat_cols = [
    'manufacturer',
    'model_grouped',
    'condition',
    'fuel',
    'title_status',
    'drive',
    'type',
    'paint_color',
    'state'
]

# Aplicar One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Ver resultado
print("Shape final del dataset:", df_encoded.shape)
df_encoded.head()

In [ ]:
# VERIFICACIÓN DE LOS CAMBIOS
df.info()

___________________________________________________________________________________________
## 6. **MODELADO**

### 6.1. **MODELADO SIMPLE**

#### 6.1.1. Modelos de valización cruzada

A modo de ir respondiendo las preguntas que se plantearon al inicio del proyecto, se realizará un diagrama de calor para ver la correlación de las variables numéricas con respecto a price:

In [ ]:
# Seleccionar solo columnas numéricas
numeric_cols = df_encoded.select_dtypes(include=['int64', 'float64']).columns

# Calcular la matriz de correlación
correlation_matrix = df_encoded[numeric_cols].corr()

# Extraer solo la fila/columna de 'price'
correlation_with_price = correlation_matrix['price'].sort_values(ascending=False)

# Mostrar top correlaciones positivas y negativas
print("🔝 Variables más positivamente correlacionadas con el precio:")
print(correlation_with_price.head(10))

print("\n🔻 Variables más negativamente correlacionadas con el precio:")
print(correlation_with_price.tail(10))

# (Opcional) Gráfico de calor completo
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix[['price']].sort_values(by='price', ascending=False), annot=True, cmap='coolwarm')
plt.title("Correlación de variables numéricas con el precio")
plt.tight_layout()
plt.show()


Como se puede visualizar, hay varias variables muy cercanas a 0, lo que indica una correlación muy baja, especialmente is_clean_title, por ejemplo, pero esto no es tan determinante porque no aportan mucho solas. En convinación con las otras variables, muy probablemente si aporten. Por otro lado, ninguna variables tiene una correlación por encima de 0,5. Esto indica que las vatiables tienen una participación en conjunto. Es decir, se reafirma lo que pasa con is_clea_tittle.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 1. Definir X e y
X = df_encoded.drop(columns=['price'])
y = df_encoded['price']

# 2. Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Entrenar modelo base
model = RandomForestRegressor(random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# 4. Predecir
y_pred = model.predict(X_test)

# 5. Métricas
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"📊 MAE:  {mae:.2f}")
print(f"📊 RMSE: {rmse:.2f}")
print(f"📊 R²:   {r2:.4f}")


#### 6.1.2. Ajuste de hiperparámetros para obtener los mejores modelos

**⚠️¡ATENCIÓN!  
El Sigueinte bloque de código corresponde a encontrar los mejores hiperparámetros usando GridSearchCV. Si no cuentas con una PC de altas prestaciones, este codigo puede demorar bastante y tener resultados que no serán los mejores.**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Definir el modelo base
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# Definir la grilla de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# Configurar el GridSearch con validación cruzada de 3 folds
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

# Ejecutar búsqueda
grid_search.fit(X_train, y_train)

# Mostrar mejores parámetros y score
print("🔍 Mejores parámetros encontrados:")
print(grid_search.best_params_)
print(f"📉 Mejor RMSE (negativo): {grid_search.best_score_:.4f}")


Una vez ejecutado el bloque de código anterior, se obtuvieron los sigueintes hiperparámetros con un RMSE (negativo) de -8920.4557:  

max_depth: **None** (los otros parámetros compensan el riesgo de overfitting)  
max_features: **log2** (subconjunto reducido de columnas)  
min_samples_leaf: **1** (las hohas del arbol tienen una sola muestra, es decir, una alta resolución)  
min_samples_split: **2** (las ramas se dividen con solo 2 muestras)  
n_estimators: **100** (se usan 100 árboles)  

Se procede a ejecutar el entrenamiento con los mejores parámetros:

In [ ]:
# 1. Definir X e y
X = df_encoded.drop(columns=['price'])
y = df_encoded['price']

# 2. Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Entrenar modelo optimizado
modelo_optimo = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    max_features='log2',
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=42
)
modelo_optimo.fit(X_train, y_train)

# 4. Predecir
y_pred = modelo_optimo.predict(X_test)

# 5. Métricas
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"📊 MAE:  {mae:.2f}")
print(f"📊 RMSE: {rmse:.2f}")
print(f"📊 R²:   {r2:.4f}")

#### 6.1.3. Trazar curvas de aprendizaje

Se debe comprender como evoluciona el rendimiento del modelo a medida que se agregan más datos al entrenamiento y luego saber si se necesitan más datos o hay un sobreajuste.

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import LearningCurveDisplay

# 1. Crear el modelo base
estimator = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

# 2. Definir el método de validación cruzada
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

# 3. Trazar la curva de aprendizaje usando R²
LearningCurveDisplay.from_estimator(
    estimator=estimator,
    X=X_train,
    y=y_train,
    cv=cv,
    scoring="r2",
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 5),  # 5 puntos de entrenamiento: 10% a 100%
)

plt.title("Curva de Aprendizaje - Random Forest (R²)")
plt.grid()
plt.tight_layout()
plt.show()


Se observa que el R² de Train es mucho mayor que Test. Esto significa que el modelo está sobreajustado (aprende demasiado bien los datos de entrenamiento y generaliza peor).
Se observa que a mayor tamaño de entrenamiento, mejora el R² de Test, es decir, más datos ayudan a generalizar mejor.
En general, el modelo tiene buena capacidad predictiva, pero hay espacio para mejorar la generalización.

#### 6.1.4. Importancia de las características (Feature Importance) de RandomForestRegressor

En RandomForestRegressor, la importancia de una característica se mide como la reducción total de error (por ejemplo, MSE) atribuible a cada feature, promediada entre todos los árboles del bosque.  
El resultado es un valor entre 0 y 1 para cada feature. Cuanto mayor el valor, más importante fue esa columna en las decisiones del modelo. Para obtener esto se debe ejecutar el siguiente bloque de código:

In [ ]:
importancia = modelo_optimo.feature_importances_
columnas = X_train.columns

# Crear un DataFrame ordenado
importancias_df = pd.DataFrame({
    'Feature': columnas,
    'Importancia': importancia
}).sort_values(by='Importancia', ascending=False)

# Mostrar las 30 características más importantes ordenadas
top_importancias = importancias_df.head(30)

for i, row in top_importancias.iterrows():
    print(f"{i+1:2d}. {row['Feature']:40s} --> {row['Importancia']:.4f}")

plt.figure(figsize=(10, 8))
plt.barh(top_importancias['Feature'][::-1], top_importancias['Importancia'][::-1], color='skyblue')
plt.xlabel("Importancia")
plt.title("Top 30 características más importantes según Random Forest")
plt.tight_layout()
plt.show()



**Conclusiones del análisis de importancia de características:**  

**Dominio de pocas variables:** odometer (0.1591), year (0.1382) y vehicle_age (0.1369) concentran una gran parte de la importancia total.  
**Desbalance de relevancia:** Desde la posición 4 en adelante, la importancia de las características cae drásticamente.   
**Alta dimensionalidad:** Muchas columnas apenas aportan entre 0.005 y 0.01, lo cual indica que son poco útiles para la predicción.  
**Redundancias:** Algunas columnas parecen variantes de una misma información. Ej: year y vehicle_age, model_grouped_unknown y model_grouped_other.  

#### 6.1.5. Mejoras en base al Feature Importance y la curva de aprendizaje

In [ ]:
# Eliminar variables de baja importancia
columnas_a_eliminar = importancias_df[importancias_df['Importancia'] < 0.005]['Feature']
X_train_filtrado = X_train.drop(columns=columnas_a_eliminar)
X_test_filtrado = X_test.drop(columns=columnas_a_eliminar)

# Eliminar columna 'year' si existe
for df in [X_train_filtrado, X_test_filtrado]:
    if 'year' in df.columns:
        df.drop(columns='year', inplace=True)

# Combinar 'model_grouped_unknown' y 'model_grouped_other' en 'model_grouped_misc'
for df in [X_train_filtrado, X_test_filtrado]:
    if 'model_grouped_unknown' in df.columns and 'model_grouped_other' in df.columns:
        df['model_grouped_misc'] = df['model_grouped_unknown'] + df['model_grouped_other']
        df.drop(columns=['model_grouped_unknown', 'model_grouped_other'], inplace=True)
    elif 'model_grouped_unknown' in df.columns:
        df.rename(columns={'model_grouped_unknown': 'model_grouped_misc'}, inplace=True)
    elif 'model_grouped_other' in df.columns:
        df.rename(columns={'model_grouped_other': 'model_grouped_misc'}, inplace=True)

Aplicaremos nuevamente RandomForestRegressor pero teniendo en cuenta de evitar el sobreajuste

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Entrenar modelo con parámetros para reducir sobreajuste
# Antes los parámetros eran:
#    n_estimators=100,
#    max_depth=None,
#    max_features='log2',
#    min_samples_leaf=1,
#    min_samples_split=2,
#    random_state=42

modelo_regularizado = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)

modelo_regularizado.fit(X_train_filtrado, y_train)

# Predicciones
y_train_pred = modelo_regularizado.predict(X_train_filtrado)
y_test_pred = modelo_regularizado.predict(X_test_filtrado)

# Métricas
mae_train = mean_absolute_error(y_train, y_train_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)

mae_test = mean_absolute_error(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)

# Mostrar resultados
print("=== Métricas de Entrenamiento ===")
print(f"MAE:  {mae_train:.2f}")
print(f"RMSE: {rmse_train:.2f}")
print(f"R²:   {r2_train:.4f}")

print("\n=== Métricas de Test ===")
print(f"MAE:  {mae_test:.2f}")
print(f"RMSE: {rmse_test:.2f}")
print(f"R²:   {r2_test:.4f}")

### 6.2. **MODELADO DE CONJUNTOS**

#### 6.2.1. Obtener un segundo modelo con otro método

Para esto, se hará uso de XGBoost. Primero se debe ejecutar *"pip install xgboost"* para instalar XGBoost en tu sistema.  
Luego se procede con el entrenamiento:

In [ ]:
from xgboost import XGBRegressor

modelo_xgb = XGBRegressor(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
modelo_xgb.fit(X_train, y_train)
y_pred_xgb = modelo_xgb.predict(X_test)


#### 6.2.2. Combinando modelos

In [ ]:
y_pred_ensemble = (y_pred_rf + y_pred_xgb) / 2

### 6.3. **PREDICCIÓN FINAL**

#### 6.3.1. Predecir y evaluar resultados

___________________________________________________________________________________________
## 7. **CONCLUSIÓN**

___________________________________________________________________________________________
## 8. **PRÓXIMAS LÍNEAS**